In [1]:
import numpy as np
import random
import itertools
import math

In [91]:
### I threat the total length of time as the length of the observered list

class HMM:
    def __init__(self,no_of_state,no_of_value,states,values):
        #randomize all matrix, each row should sum to 1
        self.emssion_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_value) for i in range(no_of_state)])]) #b[i][O]  --> probability to emit values[O] from states[i]
        
        self.initial_matrix= np.random.rand(no_of_state) #π[i] --> probability to start at states[i]
        self.initial_matrix/= self.initial_matrix.sum()
        
        self.transition_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_state) for i in range(no_of_state)])])  #a[i][j] --> probability from states[i] transit to states[j]
        
        self.states=states
        self.values=values
        self.observered=None
        
        #self.prev_initial_matrix=self.initial_matrix
        #self.prev_emssion_matrix=self.emssion_matrix
        #self.prev_transition_matrix=self.transition_matrix
        
    def debug(self):
        print('initial_matrix\n',self.initial_matrix)
        print('transition_matrix\n',self.transition_matrix)
        print('emission_matrix\n',self.emssion_matrix)
        
    def likelihood(self,state,ob):
        if ob is None:
            ob=self.observered
        prob=1
        for x in ob:
            prob*=(self.emssion_matrix[state][x])
        return prob
 
    def forward(self,t,j,ob=None):
        if ob is None:
            ob=self.observered
        if t==0:
            return self.initial_matrix[j]*self.likelihood(j,ob[t])
        else:          
            return sum([self.forward(t-1,i,ob)*self.transition_matrix[i][j] for i in range(len(self.states))]) * self.likelihood(j,ob[t])
          
            
    def backward(self,t,i,ob=None):
        if ob is None:
            ob=self.observered
        if t==len(ob)-1:
            return 1
        else:
            return sum([self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob) for j in range(len(self.states))])
           
    def probit_at_i(self,t,i,ob=None):#Gamma γt(i) = P(qt = i|O,λ)      
        if ob is None:
            ob=self.observered
        numerator=self.forward(t,i,ob)*self.backward(t,i,ob)#sum probability of all path passing through state[i] at time t
        denominator=sum([self.forward(t,j,ob)*self.backward(t,j,ob) for j in range(len(self.states))]) #prob of passing through  ALL_state at time t
        return numerator/denominator
    
    def probit_transit_i_j(self,t,i,j,ob=None):#epsilon ξt(i, j) = P(qt = i,qt+1 = j|O,λ)
        if ob is None:
            ob=self.observered
        numerator=self.forward(t,i,ob)*self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob)#sum probability of all path transit from state[i] to state[j] at time t
        denominator=sum([sum([self.forward(t,m,ob)*self.transition_matrix[m][n]*self.likelihood(n,ob[t+1])*self.backward(t+1,n,ob) for n in range(len(self.states))]) for m in range(len(self.states))]) #prob of ALL transition combination at time t
        return (numerator/denominator)
            
    def train(self,obs=None,epochs=2):
        #O:observed values
        #λ:model parameters
         
        if obs is None:
            obs=self.observered
        
        for epoch in range(epochs):
            for ob in obs:
                #initial matrix
                for i in range(len(self.states)):
                    self.initial_matrix[i]=self.probit_at_i(0,i,ob)
                #transition matrix
                for i, j in itertools.product(range(len(self.states)),range(len(self.states))):
                    self.transition_matrix[i][j]=sum([self.probit_transit_i_j(t,i,j,ob) for t in range(len(ob)-1)])/sum([self.probit_at_i(t,i,ob) for t in range(len(ob)-1)])
                #emission matrix
                for j, k in itertools.product(range(len(self.states)),range(len(self.values))):   
                    total=0
                    for t in range(len(ob)):
                        if k in ob[t]:
                            total+=self.probit_at_i(t,j,ob)
                    self.emssion_matrix[j][k]=total/sum([self.probit_at_i(t,j,ob) for t in range(len(ob))])
                    #smoothing
                    if self.emssion_matrix[j][k]==0:
                        self.emssion_matrix[j][k]=0.0000000001

In [92]:
test=HMM(2,4,["Rainy", "Sunny"],["walk", "shop", "clean","sleep"])

In [93]:
test.train([[[0], [1,0], [3,2], [0], [1], [0,1]],[[0], [0,1], [3,2], [0], [3,2], [1]],[[3], [0], [1], [2], [1], [0]]],10)

In [94]:
test.debug()

initial_matrix
 [1. 0.]
transition_matrix
 [[0.34747371 0.64853439]
 [1.         0.        ]]
emssion_matrix
 [[4.62790474e-001 3.36054960e-002 2.48911899e-001 2.48911899e-001]
 [4.49816733e-002 9.96764423e-001 9.27745040e-239 1.00000000e-010]]


## possible modification:
Different segentation method 
and
different emission calculation